In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-190405
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-190405


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "projectcluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.01,0.1,1,10),
        '--max_iter': choice(50,100,200)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
#env = Environment.get(workspace = ws, name = 'AzureML-Tutorial')
#env = Environment('user-managed-env')
#env.python.user_managed_dependencies = True

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory="./", ##Users/odl_user_188259/train.py
                      script='train.py',
                      arguments=None,
                      compute_target=cpu_cluster, #compute_config
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    run_config = src,
    hyperparameter_sampling = ps,
    policy = policy, 
    primary_metric_name = 'Accuracy',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs =  16,
    max_concurrent_runs = 4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

remote_run = exp.submit(config = hyperdrive_config, show_output = True)

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = remote_run.get_best_run_by_primary_metric()

##print("Best run details :",best_run.get_details())

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5, 
    compute_target=cpu_cluster)

In [11]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on projectcluster with default configuration
Running on remote compute: projectcluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_f782bf40-a5cc-4861-b752-8239b375b4bb,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [12]:
# Retrieve and save your best automl model.

best_model = automl_run.get_best_child()

best_model.get_details()

{'runId': 'AutoML_f782bf40-a5cc-4861-b752-8239b375b4bb_29',
 'target': 'projectcluster',
 'status': 'Completed',
 'startTimeUtc': '2022-03-29T16:49:54.037452Z',
 'endTimeUtc': '2022-03-29T16:49:58.031282Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'projectcluster\',\'subscription_id\':\'1b944a9b-fdae-4f97-aeb1-b7eea0beac53\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_f782bf40-a5cc-4861-b752-8239b375b4bb_29","experiment_name":"udacity-project","workspace_name":"quick-starts-ws-190405","subscription_id":"

In [13]:
best_model.get_metrics()

{'matthews_correlation': 0.5594602726675169,
 'norm_macro_recall': 0.5223450342058241,
 'average_precision_score_micro': 0.9817040046392798,
 'precision_score_micro': 0.9174203338391502,
 'balanced_accuracy': 0.7611725171029121,
 'accuracy': 0.9174203338391502,
 'f1_score_weighted': 0.9146086615346892,
 'log_loss': 0.23751224404089663,
 'AUC_weighted': 0.9479434429965261,
 'average_precision_score_weighted': 0.9561000170424837,
 'recall_score_weighted': 0.9174203338391502,
 'precision_score_macro': 0.7996832504732616,
 'recall_score_micro': 0.9174203338391502,
 'f1_score_macro': 0.7784667911859704,
 'f1_score_micro': 0.9174203338391503,
 'weighted_accuracy': 0.9562280319702762,
 'AUC_micro': 0.9809590242262496,
 'recall_score_macro': 0.7611725171029121,
 'average_precision_score_macro': 0.8276871868424763,
 'precision_score_weighted': 0.912802120479476,
 'AUC_macro': 0.9479434429965259,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_f782bf40-a5cc-4861-b752-8239b375b4bb_

In [14]:
best_model

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_f782bf40-a5cc-4861-b752-8239b375b4bb_29,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
# Cluster cleanup
cpu_cluster.delete()